In [414]:
from langchain_chroma import Chroma
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
from langgraph.types import interrupt, Command
from typing import TypedDict, Annotated, Literal, Optional, Dict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import operator
import json

In [541]:
# initializing the models
load_dotenv()
model = ChatGroq(
    # model=os.getenv("model")
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("api_key")
)

evaluator = ChatGroq(
    model=os.getenv("evaluater_model"),
    api_key=os.getenv("api_key")
)

embedding_model = OllamaEmbeddings(model="llama3.2")

In [542]:
loader = PyPDFLoader(file_path="Attention.pdf")
docs = loader.load()
print(len(docs))

11


In [543]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 80)
chunks = text_splitter.split_documents(docs)

print(len(chunks))

234


In [544]:
vector_store = Chroma.from_documents(
    embedding=embedding_model,
    documents=chunks,
    collection_name= "my_collection"
)

In [545]:
# here I am using mmr so that each chunk retrieved covers different angles
retrievers = vector_store.as_retriever(search_type = "mmr", search_kwargs = {'k' : 2, 'fech_k' : 5, 'lambda_mult' : 0.7})


In [546]:
class Format(BaseModel):
    output :list[str]

class Evaluation(BaseModel):
    evaluation : float = Field(ge=0.0, le=1.0, description="this containt the value got for the answer providd to the llm after evaluation.")
    feedback : str = Field(description = "this should containt the place where the refining should be done, it is like feedback to imporive the evaluation")

class Refinied(BaseModel):
    latest_question : str = Field(description= "this should contain the main questions that the other questions are made from")
    sub_questions : list[str] = Field(description= "this should contain all the questions made from decomposing the main question")

op_format = PydanticOutputParser(pydantic_object=Format)
evaluation_format = PydanticOutputParser(pydantic_object=Evaluation)
Refine_format = PydanticOutputParser(pydantic_object=Refinied)

In [547]:
class RAG_state(TypedDict):
    latest_question : Annotated[list[BaseMessage], operator.add]
    sub_questions : Annotated[list[list[str]], operator.add]
    answers : Annotated[list[list[list[str]]], operator.add]
    final_answer : str
    retry_count : int
    evaluation : float
    feedback : str
    clarification : str
    verify : str


In [548]:
# # I need sqlite db to store the checkpoints to make my agent remember the past topics
# conn = sqlite3.connect(database='RAG.db', check_same_thread=False)
# # this creates the checkpointer that saves the checkpoint in the database and allows persistance
# checkpointer = SqliteSaver(conn=conn)

# config = {'configurable' : {'thread_id': 1}}

### Query Decomposer:
- Query decomposer is used to avoid getting only the content that matches to the question and focusing on getting all the context from the document from all angles. 

 for example:

 If the question is **Why shouldn't we smoke?**, then the decomposed questions can be
 * Why somking is bad?
 * Somking effect on our health
 * How does it effect our body?

are the questions that can hit different angle, which produces the quality answers

In [549]:
# all node's task
def query_decomposition(state : RAG_state):
    # first when we get the prompt, first we decompose the main question so that we can make multiple diferent questions in the same sense to extract the relevant contnet from the RAG
    template = PromptTemplate(template="""You are a query decomposition agent.
    Your task is to transform a single high-level user question into 2–3 concise, non-overlapping sub-questions optimized for semantic search and retrieval.
    Rules:
    - Each sub-question must target a distinct angle of the original question.
    - Sub-questions must be factual, concrete, and searchable.
    - Avoid rephrasing the same question multiple times.
    - Do NOT answer the questions.
    - Do NOT add explanations.
    - Keep each sub-question under 15 words only.
    - if the question is simple then make the sub-questions simple too.
    - Prefer "what / how / why" formulations when useful.
    Question: 
    {question}
    Output format:
    {fm}""", input_variables=['question'],partial_variables={'fm' : op_format.get_format_instructions()})
    chain = template | model | op_format
    output = chain.invoke({"question" : state['latest_question'][-1]}).output
    print("Query Decomposition executed✅")
    return {'sub_questions' : [output]}

In [550]:
state = RAG_state(latest_question = ["Why is smoking bad?"])
query_decomposition(state)

Query Decomposition executed✅


{'sub_questions': [['What harms smoking cause?',
   'How smoking affects health?',
   'Why quit smoking?']]}

In [551]:
# retreiving the relevant content from the docs
def retriever(state : RAG_state):
    answers = []
    # Answer for the main question
    answer = retrievers.invoke(state['latest_question'][-1])
    answers.append(answer)
    # Answers for the sub-question
    for question in state['sub_questions'][-1]:
        answer = retrievers.invoke(question)
        answers.append(answer)
    page_content = []
    for ans in answers:
        element = []
        for i in ans:
            element.append(i.page_content)
        page_content.append(element)
    batch_answers = [page_content]
    print("Retriever executed✅")
    return {'answers' : batch_answers}

In [552]:
state = RAG_state(latest_question = ["Why is smoking bad?"], sub_questions=[['What health risks are associated with smoking?',
   'How does smoking affect the cardiovascular system?',
   'Why does smoking lead to addiction?']])
retriever(state)

Retriever executed✅


{'answers': [[['keeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.',
    'sequence (y1,...,y m) of symbols one element at a time. At each step the model is auto-regressive\n[9], consuming the previously generated symbols as additional input when generating the next.'],
   ['the maximum path length between any two input and output positions in networks composed of the\ndifferent layer types.',
    'were chosen after experimentation on the development set. We set the maximum output length during\ninference to input length + 50, but terminate early when possible [31].'],
   ['keeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.',
    'the maximum path length between any two input and output positions in networks compose

In [553]:
# # summarize the answers that we got from each sub-questions in such a way that it provides a different angle for summarizing for each list of sub-question
# def summarizer(state : RAG_state):
#     answers = state['answers'][-1]
#     question = state['latest_question'][-1]
#     sub_questions = state['sub_questions'][-1]
#     ques_ans_dict : dict[str, list[str]] = {}
#     ques_ans_dict[question] = answers[0]
#     for index, answer in enumerate(answers):
#         if index == 0:
#             continue
#         ques_ans_dict[sub_questions[index-1]] = answer
#     ques_ans_dict_str = json.dumps(ques_ans_dict, indent=2)
#     print(type(ques_ans_dict_str))
#     template = PromptTemplate(template="""You are a summarization agent.
#     Task:
#     - Summarize the retrieved chunks for each sub-question.
#     - Each sub-question maps to a list of concise summary sentences.
#     - Summaries may mix information from multiple chunks, but must reflect the original content accurately.
#     - Do not hallucinate; use only the content from the chunks.
#     - Do not dilute important details.       
#     - If there is no context in the answer list based on the question then just say "No context match". 
                                                                                                              
#     Input:
#     - questions_and_dict: a JSON object where each key is a sub-question, and each value is a list of strings (chunks retrieved for that sub-question).\n
#     {ques_ans_dict_str}
                                                
#     Output format:
#     The output should only contain the answer only, pure string
#     {format_ins}
#     """, input_variables = ['ques_ans_dict_str'], partial_variables={'format_ins' : summary_format.get_format_instructions()})
#     chain = template | model | summary_format
#     result = chain.invoke({"ques_ans_dict_str" : ques_ans_dict_str}).summary
#     print("Summarizer executed✅")
#     print(result)
#     return {'final_answer' : result}

In [554]:
import json

def summarizer(state: RAG_state):
    latest_question = state['latest_question'][-1]
    sub_questions = state['sub_questions'][-1]
    all_answers = state['answers'][-1]  # list of lists of strings

    # Build dict mapping question -> list of answer chunks
    ques_ans_dict: dict[str, list[str]] = {}
    for i, q in enumerate([latest_question] + sub_questions):
        if i < len(all_answers):
            ques_ans_dict[q] = all_answers[i]
        else:
            ques_ans_dict[q] = ["No context match"]

    # Serialize dict to JSON string for the prompt
    ques_ans_dict_str = json.dumps(ques_ans_dict, indent=2)
    template = PromptTemplate(
        template="""You are an agent that provides me answer from the sub_question and chunks provided to you.
Rules:
- Summarize the retrieved chunks for each sub-question.
- Summaries may mix information from multiple chunks, but must reflect the original content accurately.
- Do not hallucinate; use only the content from the chunks.
- Do not dilute important details.
- Give me the answer in aggregate not for each sub-question
Summarize on the basis of the given information: \n{ques_ans_dict}

**If there is no context in the answer list based on the question then just say "No context match" and nothing else**
""",
        input_variables=['ques_ans_dict'])

    chain = template | model | StrOutputParser()
    result = chain.invoke({"ques_ans_dict": ques_ans_dict_str})
    print("Summarizer executed ✅")
    return {'final_answer': result}


In [555]:
state = RAG_state(latest_question = ["Why is smoking bad?"], 
                  sub_questions=[['What health risks are associated with smoking?', 'How does smoking affect the cardiovascular system?','Why does smoking lead to addiction?']], 
                  answers=[[['keeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.',
                            'heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\nand semantic structure of the sentences.\n5 Training'],
                            ['the maximum path length between any two input and output positions in networks composed of the\ndifferent layer types.',
                            'were chosen after experimentation on the development set. We set the maximum output length during\ninference to input length + 50, but terminate early when possible [31].'],
                            ['keeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.',
                            'aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate\nself-attention and discuss its advantages over models such as [14, 15] and [8].\n3 Model Architecture'],
                            ['the input or output sequences [2, 16]. In all but a few cases [22], however, such attention mechanisms\nare used in conjunction with a recurrent network.',
                            'nov. Dropout: a simple way to prevent neural networks from overﬁtting. Journal of Machine\nLearning Research, 15(1):1929–1958, 2014.']]])
summarizer(state)

Summarizer executed ✅


{'final_answer': 'No context match'}

In [556]:
# evaluating the asnwer based on the question and sub_questions generated, if the evaluation is > 0.7 then pass and if still better can be done then retry after refining the query and clarification
def evaluation(state : RAG_state):
    latest_question = state['latest_question'][-1]
    questions = state['sub_questions'][-1]
    final_answer = state['final_answer']
    prompt = PromptTemplate(template="""You are an evaluation agent.
    Task:
    - Evaluate the quality of the provided answer to the given question.
    - Provide a **score between 0 and 1**, where 1 is perfect and 0 is completely wrong.
    - Provide **feedback** explaining why the score was given and suggestions to improve the answer (e.g., missing information, inaccurate, vague, unclear wording).
    - Be concise but informative in the feedback.
    - Base your evaluation **strictly on the question and the provided answer**. Do not assume extra knowledge.
    - Do NOT include the question in the output, only the score and feedback.

    Input:
    - question: the original user question
    {latest_question} \n
    {sub_questions}
    - answer: the generated answer to evaluate
    {final_answer}                            
    Output format (return only valid JSON):
    {format_ins}                
    """, input_variables=['latest_question', 'sub_questions', 'final_answer'], partial_variables={'format_ins': evaluation_format.get_format_instructions()})
    chain = prompt | evaluator | evaluation_format
    result = chain.invoke({'latest_question' : latest_question, "sub_questions" : questions, "final_answer" : final_answer})
    print("Evaluation executed✅")
    return {"evaluation" : result.evaluation, "feedback" : result.feedback}

In [557]:
state = RAG_state(latest_question = ["Why is smoking bad?"], 
                  sub_questions=[['What health risks are associated with smoking?', 'How does smoking affect the cardiovascular system?','Why does smoking lead to addiction?']], 
                  final_answer= 'No context match')

evaluation(state)

Evaluation executed✅


{'evaluation': 0.0,
 'feedback': 'The answer does not provide any relevant information to address the question. It seems to be a placeholder or an error message, indicating that there is no context match. To improve, the answer should provide a clear and concise explanation of why smoking is bad, including its health risks and effects on the body.'}

In [575]:
# we need conditional function for conditional workflow for either to retry or ask for user clarification
def evaluation_conditional_edge(state : RAG_state):
    if state['retry_count'] >2 or state['final_answer'] == "No context match":
        return "conclude"
    elif state['evaluation'] <= 0.5:
        print("retry count➿: ", state['retry_count'])
        return "refine"
    else:
        return "clarify"

def human_conditional_edge(state: RAG_state):
    if state['verify'] == True:
        return END
    return 'refine'

In [576]:
# refining the query by focusing on the way the evaluater has guided and human clarification(if required)
def refining_query(state: RAG_state):
    # increasing the rety count as refining is retry
    latest_question = state['latest_question'][-1]
    sub_questions = state['sub_questions'][-1]
    final_answer = state['final_answer']
    feedback = state['feedback']
    if state.get('clarification'):
        feedback = {
            'feedback_during_evaluation' : feedback,
            'clarification_from_user' : state['clarification']
        }
    template = PromptTemplate(template="""You are a query refinement agent.
    Your task:
    - Refine the original main question using the provided feedback.
    - Generate 2–3 refined sub-questions optimized for semantic search and document retrieval.
    - Improve clarity, specificity, and retrievability.
    - Fix issues mentioned in the feedback (e.g., vagueness, missing aspects, incorrect focus).

    Inputs:
    - original_main_question: {latest_question}
    - original_sub_questions: {sub_questions}
    - feedback: {feedback}
    - final_answer : {final_answer}

    Rules:
    - Do NOT answer any question.
    - Do NOT add explanations.
    - Avoid repeating the original questions unless necessary.
    - Each sub-question must target a distinct aspect.
    - Each sub-question must be factual, concrete, and searchable.
    - Keep sub-questions under 20 words.
    - Prefer "what / how / why" formulations where appropriate.

    Output format:
    Return ONLY valid JSON in this exact structure: 
    {format_ins}
    """, input_variables=['latest_question', 'sub_questions', 'feedback', 'final_answer'], partial_variables={'format_ins' : Refine_format.get_format_instructions()})  

    chain = template | model | Refine_format
    result = chain.invoke({'latest_question' : latest_question, 'sub_questions' : sub_questions, 'feedback' : feedback, 'final_answer': final_answer})
    print("RefiningQuery executed✅")
    return {'latest_question' : [result.latest_question], 'sub_questions' : [result.sub_questions], 'retry_count' : state['retry_count'] + 1}

In [577]:
state = RAG_state(latest_question = ["Why is smoking bad?"], 
                  sub_questions=[['What health risks are associated with smoking?', 'How does smoking affect the cardiovascular system?','Why does smoking lead to addiction?']], 
                  final_answer= 'No context match',
                  evaluation= 0.0,
                  feedback= "The answer does not provide any relevant information to address the question. It simply states 'No context match' without explaining why or providing any context. To improve, the answer should directly address the question, providing clear and concise information about the health risks associated with smoking.",
                  retry_count=1
                  )
refining_query(state)

RefiningQuery executed✅


{'latest_question': ['What are the health effects of smoking?'],
 'sub_questions': [['What diseases are linked to smoking?',
   'How does smoking harm lungs?',
   'What toxins are in cigarette smoke?']],
 'retry_count': 2}

In [578]:
# human clarification for the asnwer and the question
def human_clarification(state : RAG_state):
    decision = interrupt({
        "type" : "preview",
        "reason" : "Agent need to know user's clarification",
        "instruction" : "If Approve the answer then Yes, Reject then give feedback where to point more or clarify more"
    })
    if decision == "yes":
        state["verify"] = True
    feedback_from_user = decision
    print("HumanClarification executed✅")
    return {"clarification" : feedback_from_user}

In [579]:
def conclusion(state: RAG_state):
    return {'final_answer' : state['final_answer']}

In [581]:
graph = StateGraph(RAG_state)

graph.add_node("QueryDecomposition", query_decomposition)
graph.add_node("Retriever", retriever)
graph.add_node("Summarizer", summarizer)
graph.add_node("Evaluation", evaluation)
graph.add_node("RefiningQuery", refining_query)
graph.add_node("HumanClarification", human_clarification)
graph.add_node("Conclusion", conclusion)

graph.add_edge(START, "QueryDecomposition")
graph.add_edge("QueryDecomposition", "Retriever")
graph.add_edge("Retriever","Summarizer")
graph.add_edge("Summarizer", "Evaluation")
graph.add_conditional_edges("Evaluation", evaluation_conditional_edge, {'refine' : "RefiningQuery", 'clarify' : "HumanClarification", "conclude" : "Conclusion"})
graph.add_conditional_edges("HumanClarification", human_conditional_edge, {'accept' : 'Conclusion', 'reject' : "RefiningQuery"})
graph.add_edge("RefiningQuery", "QueryDecomposition")
graph.add_edge("Conclusion", END)


workflow = graph.compile()
# workflow

In [582]:
workflow.invoke({"latest_question" : ["Why is smoking bad?"], 'retry_count' : 0})

Query Decomposition executed✅
Retriever executed✅
Summarizer executed ✅
Evaluation executed✅


{'latest_question': ['Why is smoking bad?'],
 'sub_questions': [['What harms smoking causes',
   'How smoking affects health',
   'Why quit smoking']],
 'answers': [[['keeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.',
    'sequence (y1,...,y m) of symbols one element at a time. At each step the model is auto-regressive\n[9], consuming the previously generated symbols as additional input when generating the next.'],
   ['attention and the parameter-free position representation and became the other person involved in nearly every',
    'heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\nand semantic structure of the sentences.\n5 Training'],
   ['versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version\nbecause it may allow the model to extrapolate to seq